In [607]:
#!pip install torch
import sqlite3
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
import pandas as pd
import numpy as np

In [608]:
import torch
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import random_split

# Подготовка данных

In [609]:
# Создаем соединение с базой данных
conn = sqlite3.connect('garpix_db_0.0.2.db')

In [610]:
# Формируем DataFrame  
df = pd.read_sql_query("""
    SELECT
        dr.calculation_id,
		b.size_width, 
		b.size_height, 
		b.size_length,
		dr.density_percent,
		dr.filling_space_percent
    FROM data_result dr
    JOIN boxes b ON b.calculation_id = dr.calculation_id
""", conn)
print(df.head())

   calculation_id  size_width  size_height  size_length  density_percent  \
0           16900        98.0        143.0        385.0         62.12352   
1           16900       196.0        198.0        363.0         62.12352   
2           16900       277.0        190.0        407.0         62.12352   
3           16900       170.0        325.0        255.0         62.12352   
4           16900       250.0        350.0        160.0         62.12352   

   filling_space_percent  
0               61.22618  
1               61.22618  
2               61.22618  
3               61.22618  
4               61.22618  


In [611]:
grouped_df = df.groupby('calculation_id').agg(list)
grouped_df.groupby('calculation_id').agg(pd.Series.tolist)

,size_width,size_height,size_length,density_percent,filling_space_percent
calculation_id,,,,,
11834,"[[196.0, 297.0, 297.0, 57.0, 96.0, 92.0, 57.0,...","[[311.0, 279.0, 279.0, 117.0, 194.0, 202.0, 11...","[[324.0, 387.0, 387.0, 182.0, 141.0, 188.0, 18...","[[81.76416, 81.76416, 81.76416, 81.76416, 81.7...","[[81.72327, 81.72327, 81.72327, 81.72327, 81.7..."
12729,"[[300.0, 300.0, 300.0, 300.0, 300.0, 300.0, 30...","[[250.0, 250.0, 250.0, 250.0, 250.0, 250.0, 25...","[[400.0, 400.0, 400.0, 400.0, 400.0, 400.0, 40...","[[82.5, 82.5, 82.5, 82.5, 82.5, 82.5, 82.5, 82...","[[51.5625, 51.5625, 51.5625, 51.5625, 51.5625,..."
13579,"[[800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 80...","[[1550.0, 1550.0, 1550.0, 1550.0, 1550.0, 1550...","[[1200.0, 1200.0, 1200.0, 1200.0, 1200.0, 1200...","[[94.44444, 94.44444, 94.44444, 94.44444, 94.4...","[[64.24381, 64.24381, 64.24381, 64.24381, 64.2..."
13580,"[[300.0, 300.0, 300.0, 200.0, 300.0, 300.0, 30...","[[300.0, 300.0, 300.0, 200.0, 300.0, 300.0, 30...","[[500.0, 500.0, 500.0, 400.0, 500.0, 500.0, 50...","[[74.02778, 74.02778, 74.02778, 74.02778, 74.0...","[[67.29798, 67.29798, 67.29798, 67.29798, 67.2..."
14208,"[[800.0, 800.0, 800.0, 800.0, 800.0, 800.0, 80...","[[1600.0, 1600.0, 1600.0, 1600.0, 1600.0, 1600...","[[1200.0, 1200.0, 1200.0, 1200.0, 1200.0, 1200...","[[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 10...","[[58.26317, 58.26317, 58.26317, 58.26317, 58.2..."
...,...,...,...,...,...
17374,"[[986.0, 966.0, 800.0, 908.0, 800.0, 800.0, 88...","[[2095.0, 2095.0, 2095.0, 2095.0, 2095.0, 2095...","[[1396.0, 1326.0, 1200.0, 1200.0, 1200.0, 1200...","[[91.90237, 91.90237, 91.90237, 91.90237, 91.9...","[[76.74275, 76.74275, 76.74275, 76.74275, 76.7..."
17375,"[[908.0, 908.0, 883.0, 883.0, 823.0, 828.0, 82...","[[2095.0, 2095.0, 2095.0, 2095.0, 2095.0, 2095...","[[1200.0, 1200.0, 1200.0, 1200.0, 1200.0, 1200...","[[97.00727, 97.00727, 97.00727, 97.00727, 97.0...","[[83.18305, 83.18305, 83.18305, 83.18305, 83.1..."
17567,[[100.0]],[[100.0]],[[100.0]],[[100.0]],[[0.00248]]


In [612]:
# Группируем данные по calculate_id 
# Группируем DataFrame по calculate_id и создаем новый столбец с именем "width_list", 
# который содержит список всех значений size_width (ширина ящика) для каждой группы. 
grouped_df = df.groupby('calculation_id')[['size_width', 'size_height', 'size_length']].agg(list)
#.apply(list)
#.apply(list).reset_index(name='width_list')
grouped_df['data_1'] = grouped_df['size_width'] + grouped_df['size_height'] + grouped_df['size_length']
grouped_df['data_2'] = grouped_df['size_width'] + grouped_df['size_length'] + grouped_df['size_height']
grouped_df['data_3'] = grouped_df['size_height'] + grouped_df['size_width'] + grouped_df['size_length']
grouped_df['data_4'] = grouped_df['size_height'] + grouped_df['size_length'] + grouped_df['size_width']
grouped_df['data_5'] = grouped_df['size_length'] + grouped_df['size_height'] + grouped_df['size_width']
grouped_df['data_6'] = grouped_df['size_length'] + grouped_df['size_width'] + grouped_df['size_height']
data_df_1 = grouped_df['data_1']
data_df_2 = grouped_df['data_2']
data_df_3 = grouped_df['data_3']
data_df_4 = grouped_df['data_4']
data_df_5 = grouped_df['data_5']
data_df_6 = grouped_df['data_6']

In [613]:
temp_index = [index for index in grouped_df['size_width'].index]
temp_index

[11834,
 12729,
 13579,
 13580,
 14208,
 14457,
 14770,
 14771,
 14772,
 14775,
 14781,
 14784,
 14786,
 14787,
 14793,
 14798,
 14803,
 14806,
 14808,
 14809,
 14810,
 14811,
 14836,
 14860,
 14864,
 14870,
 14905,
 14933,
 14941,
 14942,
 14946,
 14947,
 14948,
 14949,
 14950,
 14951,
 14952,
 14955,
 14957,
 14974,
 14975,
 14993,
 15024,
 15044,
 15045,
 15046,
 15073,
 15077,
 15078,
 15079,
 15080,
 15082,
 15083,
 15129,
 15134,
 15135,
 15140,
 15159,
 15160,
 15161,
 15173,
 15174,
 15183,
 15186,
 15190,
 15198,
 15211,
 15212,
 15215,
 15216,
 15217,
 15234,
 15253,
 15254,
 15255,
 15275,
 15277,
 15297,
 15313,
 15316,
 15317,
 15318,
 15323,
 15324,
 15325,
 15326,
 15327,
 15328,
 15335,
 15340,
 15341,
 15342,
 15346,
 15349,
 15355,
 15361,
 15363,
 15364,
 15372,
 15374,
 15380,
 15386,
 15388,
 15389,
 15391,
 15398,
 15399,
 15400,
 15402,
 15403,
 15404,
 15405,
 15406,
 15407,
 15408,
 15411,
 15412,
 15421,
 15428,
 15429,
 15432,
 15433,
 15435,
 15437,
 15440,


In [614]:
def get_data_index_1(index):
	temp_1 = []
	len_temp = len(grouped_df['size_width'][index])
	for len_x in range(len_temp):
		temp_1 = temp_1 + [grouped_df['size_width'][index][len_x], grouped_df['size_height'][index][len_x], grouped_df['size_length'][index][len_x]]
	return temp_1

def get_data_index_2(index):
	temp_1 = []
	len_temp = len(grouped_df['size_width'][index])
	for len_x in range(len_temp):
		temp_1 = temp_1 + [grouped_df['size_length'][index][len_x], grouped_df['size_height'][index][len_x], grouped_df['size_width'][index][len_x]]
	return temp_1

def get_data_index_3(index):
	temp_1 = []
	len_temp = len(grouped_df['size_width'][index])
	for len_x in range(len_temp):
		temp_1 = temp_1 + [grouped_df['size_height'][index][len_x], grouped_df['size_width'][index][len_x], grouped_df['size_length'][index][len_x]]
	return temp_1

In [615]:
# Наконец, создаем список тензоров из этих списков значений size_width.
data_x =  [torch.tensor(get_data_index_1(index)) for index in temp_index] # + \
    #      [torch.tensor(get_data_index_2(index)) for index in temp_index] + \
     #     [torch.tensor(get_data_index_3(index)) for index in temp_index]
for x in range(0, len(data_x)):
	data_x[x] = data_x[x].to(torch.float)
	data_x[x] = data_x[x] / data_x[x].max()


In [616]:
data_y = pad_sequence(data_x, batch_first=True)

In [617]:
resuld_df = df.groupby('calculation_id')['density_percent'].agg(list)

In [618]:
# Создаем np.array уникальных значений filling_space_percent для каждого calculation_id
result =  [[x[0]] for x in resuld_df.values]# + [[x[0]] for x in resuld_df.values] + [[x[0]] for x in resuld_df.values]
result_data = np.array(result)
for x in range(0, len(result_data)):
	result_data[x] = result_data[x]
	result_data[x] = result_data[x] / 100.
result_data

array([[0.8176416],
       [0.825    ],
       [0.9444444],
       [0.7402778],
       [1.       ],
       [0.9220825],
       [0.5572132],
       [0.827095 ],
       [0.5869138],
       [0.5572132],
       [1.       ],
       [0.953125 ],
       [0.9943182],
       [0.9943182],
       [0.8063824],
       [0.6338312],
       [0.6254826],
       [0.476821 ],
       [0.476821 ],
       [0.4993126],
       [0.4780008],
       [0.4937232],
       [0.555809 ],
       [0.8915496],
       [0.7615648],
       [0.7615648],
       [0.9943182],
       [0.8727273],
       [0.9051218],
       [0.9051218],
       [1.       ],
       [0.82084  ],
       [0.7633322],
       [0.7912946],
       [0.8695652],
       [0.6884688],
       [0.4511657],
       [0.9464265],
       [0.9464265],
       [0.7910819],
       [0.6364827],
       [0.9824176],
       [0.486333 ],
       [0.4851001],
       [0.4851001],
       [0.4851001],
       [0.7612721],
       [0.8859972],
       [0.6750455],
       [0.7912094],


# Подготовка модели

In [619]:
# Определяется модель Net - это нейронная сеть, 
# состоящая из слоя RNN (рекуррентная нейронная сеть) и полносвязного слоя. 
# Скрытый размерность RNN равен 16, а входной размерность равна 1.
# Модель использует функцию активации по умолчанию.
class Net(nn.Module):
# В конструкторе класса мы определяем наши слои: nn.RNN и nn.Linear. 
# Эти слои будут использоваться при проходе данных через модель. 
# Здесь мы указываем, что на входе имеем 1 признак, 
# размер скрытого состояния RNN равен 16, 
# а архитектура сети предполагает использование batch_first=True, 
# то есть размерность первой оси является размером пакета.
    def __init__(self):
        super(Net, self).__init__()
        self.rnn = nn.RNN(input_size=1, hidden_size=8, batch_first=True)
        self.fc = nn.Linear(8, 1)

# Функция forward принимает входной тензор x, 
# пропускает его через слой RNN, 
# получает выходной тензор и пропускает его через полносвязный слой, 
# чтобы получить выходной тензор.
    def forward(self, x):
        output, _ = self.rnn(x)
        x = self.fc(output[:, -1, :])
        return x.squeeze()  # удаляем избыточные размерности

In [620]:
model = Net()
# определяется устройство, на котором будет проходить обучение модели (GPU или CPU).
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = "cpu"
model.to(device)

Net(
  (rnn): RNN(1, 8, batch_first=True)
  (fc): Linear(in_features=8, out_features=1, bias=True)
)

In [621]:
device

device(type='cuda')

In [622]:
# Определение функции потерь и оптимизатора
# Определяется функция потерь (criterion) - это среднеквадратичная ошибка (MSE).
#criterion = nn.MSELoss()
criterion = nn.L1Loss()
# Определяется оптимизатор (optimizer) - 
# это алгоритм Adam с коэффициентом обучения 0.001, 
# который будет использоваться для обновления параметров модели.
#optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [623]:
import numpy as np
num_samples = 1000
data = []
for i in range(len(data_x)):
    tmp = torch.from_numpy(data_y[i].numpy()).reshape(8547,1)
    data.append(tmp.float().to(device),)

In [624]:
# Генерируется синтетический набор данных для обучения - это список data, 
# который состоит из 1000 тензоров, 
# каждый из которых имеет случайный размер от 3 до 10 и 
# содержит случайные числа от 0 до 100. 
# Каждый тензор data заполняется нулями до максимальной длины, 
# чтобы все тензоры имели одинаковую длину. 
# Кроме того, создается тензор меток labels, 
# который состоит из 1000 случайных чисел от 0 до 100.
data_tensor = pad_sequence(data, batch_first=True)  # заполняем более короткие тензоры нулями

In [625]:
data_tensor.to(device)

tensor([[[0.4925],
         [0.7814],
         [0.8141],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.7500],
         [0.6250],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[0.5161],
         [1.0000],
         [0.7742],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        ...,

        [[1.0000],
         [1.0000],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[1.0000],
         [1.0000],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]],

        [[1.0000],
         [1.0000],
         [1.0000],
         ...,
         [0.0000],
         [0.0000],
         [0.0000]]], device='cuda:0')

In [626]:
# перемещение labels на устройство обусловлено тем, 
# что он используется в вычислении функции потерь, 
# которая определена на устройстве.
labels = torch.from_numpy(result_data).float().to(device)

In [627]:
labels

tensor([[0.8176],
        [0.8250],
        [0.9444],
        [0.7403],
        [1.0000],
        [0.9221],
        [0.5572],
        [0.8271],
        [0.5869],
        [0.5572],
        [1.0000],
        [0.9531],
        [0.9943],
        [0.9943],
        [0.8064],
        [0.6338],
        [0.6255],
        [0.4768],
        [0.4768],
        [0.4993],
        [0.4780],
        [0.4937],
        [0.5558],
        [0.8915],
        [0.7616],
        [0.7616],
        [0.9943],
        [0.8727],
        [0.9051],
        [0.9051],
        [1.0000],
        [0.8208],
        [0.7633],
        [0.7913],
        [0.8696],
        [0.6885],
        [0.4512],
        [0.9464],
        [0.9464],
        [0.7911],
        [0.6365],
        [0.9824],
        [0.4863],
        [0.4851],
        [0.4851],
        [0.4851],
        [0.7613],
        [0.8860],
        [0.6750],
        [0.7912],
        [0.7912],
        [0.7964],
        [0.5507],
        [0.8333],
        [0.6200],
        [1

In [628]:
# изменяем размерность целевого тензора
# Аргумент -1 в данном случае означает, 
# что размерность должна быть вычислена автоматически 
# на основе других размерностей тензора.
labels = labels.view(-1)

In [629]:
train_size = int(0.8 * len(data_tensor))
test_size = len(data_tensor) - train_size

In [630]:
train_dataset = data_tensor[0:train_size]
test_dataset = data_tensor[train_size:]


In [631]:
target = labels[0:train_size]
test_target = labels[train_size:]

In [632]:
import statistics

In [633]:
criterion = nn.MSELoss()

In [634]:
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [635]:
# обучение модели - модель пропускает каждый тензор из data через себя, 
# получая предсказание output, 
# затем сравнивает его с соответствующей меткой target, 
# используя функцию потерь criterion, и вычисляет ошибку loss. 
# Далее выполняется обратное распространение ошибки и обновление весов optimizer.step(). 
# Процесс обучения повторяется в цикле 100 раз.
epoch = 0
for epoch in range(1000):
#while (1):
# Обнуление градиента всех параметров модели. 
# Это необходимо делать перед каждым проходом (эпохой) обучения модели, 
# чтобы избежать накопления градиентов от предыдущих проходов. 
# Без этого может произойти неожиданное обновление весов, 
# которое может ухудшить качество модели.
    optimizer.zero_grad()

# Вызываем метод forward модели model с тензором data_tensor в качестве входных данных
    output = model(train_dataset)
    #target = labels

# Вычисляем функцию потерь между предсказаниями модели output[-1] и истинными значениями target.
    loss = criterion(output, target)

# Выполняем обратное распространение ошибки и обновляем веса
	# выполняет обратное распространение ошибки (backpropagation) в нейронной сети. 
	# Она вычисляет градиенты функции потерь по всем параметрам модели
    loss.backward()
	# обновление весов модели с помощью оптимизатора 
    optimizer.step()

# loss.item() — значению функции потерь на данной эпохе. 
# Функция потерь (MSE) измеряет разницу между прогнозируемым значением и 
# фактическим значением, поэтому чем меньше значение функции потерь, 
# тем лучше модель обучена на данном этапе
    if epoch % 1 == 0:
        tmp = abs(output - target)*100
        it = [x.item() for x in tmp ]
        print('Epoch {}, Loss {}, max {}, mean {}, min {}, median {}'.format(epoch, loss.item(), max(it), statistics.mean(it), min(it), statistics.median(it)))
        torch.save(model, 'model_' + str(epoch) + '_' + str(loss.item()) + '.pt')

Epoch 0, Loss 0.76043301820755, max 109.78269958496094, mean 85.28201596765405, min 21.75774574279785, median 87.83881378173828
Epoch 1, Loss 0.7497093081474304, max 109.15162658691406, mean 84.65096233185544, min 21.12668228149414, median 87.20775604248047
Epoch 2, Loss 0.7391316294670105, max 108.5245132446289, mean 84.02385493578664, min 20.499561309814453, median 86.58063507080078
Epoch 3, Loss 0.7286987900733948, max 107.90135955810547, mean 83.4007121614251, min 19.876407623291016, median 85.95748138427734
Epoch 4, Loss 0.7184080481529236, max 107.2821044921875, mean 82.7814676619146, min 19.25715446472168, median 85.33822631835938
Epoch 5, Loss 0.7082575559616089, max 106.66670989990234, mean 82.1660842173603, min 18.64175796508789, median 84.72283172607422
Epoch 6, Loss 0.6982442736625671, max 106.05509185791016, mean 81.55447487622143, min 18.03013801574707, median 84.1112060546875
Epoch 7, Loss 0.6883668303489685, max 105.44723510742188, mean 80.94663432299853, min 17.4222850

In [636]:
torch.save(model, 'model_result_6.pt')

In [637]:
model = torch.load('model_result_5.pt')

In [638]:
# compute the loss (mean squared error)
output = model(test_dataset)
loss = criterion(output, test_target)
# output.backward()
res = abs(output - test_target)*100
#print('max = ' + str(res.max))
result = [x.item() for x in res ]
print('MSE loss: {}, max {}, mean {}, min {}, median {}'.format(loss.item(), max(result), statistics.mean(result), min(result), statistics.median(result)))

MSE loss: 0.03394811227917671, max 56.17231750488281, mean 16.15545492134397, min 0.5132496356964111, median 16.72059917449951


In [639]:
sorted(result)[:-20:-1]

[56.17231750488281,
 34.97474670410156,
 34.37070846557617,
 34.309898376464844,
 28.77303695678711,
 28.3837890625,
 27.783519744873047,
 27.365076065063477,
 27.163448333740234,
 26.86467742919922,
 24.49582290649414,
 24.49582290649414,
 24.49582290649414,
 24.49582290649414,
 24.49582290649414,
 24.49582290649414,
 24.49582290649414,
 24.49582290649414,
 24.49582290649414]

In [640]:
sorted(result)[:20:]

[0.5132496356964111,
 0.682145357131958,
 1.0119497776031494,
 1.0119497776031494,
 1.1591136455535889,
 1.2061893939971924,
 1.7673313617706299,
 1.7673313617706299,
 1.7673313617706299,
 2.3431718349456787,
 2.3431718349456787,
 2.7455687522888184,
 3.037494421005249,
 3.7365972995758057,
 4.358631134033203,
 4.378378391265869,
 4.422175884246826,
 5.392664909362793,
 6.743949890136719,
 6.98060417175293]

In [641]:
criterion = nn.CrossEntropyLoss()
loss = criterion(output, test_target)
print("CrossEntropyLoss loss:", loss)

CrossEntropyLoss loss: tensor(484.3229, device='cuda:0', grad_fn=<DivBackward1>)


In [642]:
criterion = nn.PoissonNLLLoss()
loss = criterion(output, test_target)
print("PoissonNLLLoss loss:", loss)

PoissonNLLLoss loss: tensor(1.5276, device='cuda:0', grad_fn=<MeanBackward0>)


In [643]:
criterion = nn.BCELoss()
loss = criterion(output, test_target)
print("BCELoss loss:", loss)

BCELoss loss: tensor(0.5120, device='cuda:0', grad_fn=<BinaryCrossEntropyBackward0>)


In [644]:
criterion = nn.BCEWithLogitsLoss()
loss = criterion(output, test_target)
print("BCEWithLogitsLoss loss:", loss)

BCEWithLogitsLoss loss: tensor(0.5402, device='cuda:0',
       grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)
